In [6]:
import openai
import pandas as pd
import json
from IPython.display import Markdown
import sys

In [2]:
zz=openai.Model.list()

In [3]:
len(zz['data'])

62

In [4]:
[zz['data'][idx]['id'] for idx in range(len(zz['data'])) if 'gpt' in zz['data'][idx]['id']]

['gpt-4-0314',
 'gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo',
 'gpt-3.5-turbo-0613',
 'gpt-4',
 'gpt-4-0613']

In [5]:
model_params = {
    'model': "gpt-4",
    'temperature': 0,,
}

In [7]:
def print_save(string, file='../cmp2.md'):
    print(string)
    with open(file,'w') as f:
        f.write(string)

## Load `prompt_template.md`

In [8]:
sys.path.append('..')
from utils import *
from functools import partial

In [9]:
# prompt_dict=load_prompt_template('../prompt_template.md')
prompt_dict = partial(load_prompt_template,file='../prompt_template.md')

In [9]:
prompt_dict().keys()

dict_keys(['Preamble', 'Construct Kinetic Hamiltonian (continuum version, single-particle)', 'Construct Kinetic Hamiltonian (continuum version, second-quantized)', 'Construct Kinetic Hamiltonian (lattice version)', 'Define each term in Kinetic Hamiltonian (continuum version)', 'Construct Potential Hamiltonian (continuum version)', 'Define each term in Potential Hamiltonian (continuum version)', 'Construct interaction Hamiltonian (real space, lattice version)', 'Construct interaction Hamiltonian (momentum space)', 'Convert from single-particle to second-quantized form, return in matrix', 'Convert from single-particle to second-quantized form, return in summation (expand the matrix)', 'Convert noninteracting Hamiltonian in real space to momentum space (continuum version)', 'Convert noninteracting Hamiltonian in real space to momentum space (lattice version)', 'Convert interacting Hamiltonian in real space to momentum space (lattice version)', 'Particle-hole transformation', 'Simplify the

In [8]:
with open('1106.6060.jsonl','r') as f:
    kwargs= [json.loads(line) for line in f]


In [9]:
import re
import string
def format_check(s, kwargs):
    placeholders=  [v[1] for v in (string.Formatter().parse(s)) if v[1] is not None]
    extra_args = set(kwargs.keys()) - set(placeholders+ ['task','answer'])  # Check for extra args.
    return extra_args


In [ ]:
[format_check(prompt_dict[kwarg['task']],kwarg) for kwarg in kwargs]

In [11]:
df=pd.DataFrame(list(prompt_dict().items()),columns=['Task','Prompt'])

## Construct Kinetic Hamiltonian (lattice version)

In [10]:
user_msg=[]
AI_msg=[]
sys_msg=[{'role': 'system', 'content': prompt_dict()['Preamble']}]
kwargs_list=[]

In [11]:
kwargs = {
    'task': 'Construct Kinetic Hamiltonian (lattice version)',
    'system': 'CuO$_2$',
    'real|momentum': 'real',
    'single-particle|second-quantized': 'second-quantized',
    'dof': r'spin index (spin up and spin down), orbital index ($d$ orbital in Cu atom, $p_x$ orbital in O atom, and $p_y$ orbital in O atom)',
    'site i and site j with the amplitude hopping': r'''1. $c_{d,s}^\dagger(R_i)$ and $c_{p_x,s}^\dagger(R_i+\hat{x}/2)$ with the hopping amplitude $-t_pd$.  
2. $c_{d,s}^\dagger(R_i)$ and $c_{p_x,s}^\dagger(R_i-\hat{x}/2)$ with the hopping amplitude $-t_pd$.  
3. $c_{d,s}^\dagger(R_i)$ and $c_{p_y,s}^\dagger(R_i+\hat{y}/2)$ with the hopping amplitude $-t_pd$.  
4. $c_{d,s}^\dagger(R_i)$ and $c_{p_y,s}^\dagger(R_i-\hat{y}/2)$ with the hopping amplitude $-t_pd$.  
5. $c_{p_x,s}^\dagger(R_i+\hat{x}/2)$ and $c_{p_y,s}^\dagger(R_i+\hat{y}/2)$ with the hopping amplitude $-t_pp$.  
6. $c_{p_x,s}^\dagger(R_i+\hat{x}/2)$ and $c_{p_y,s}^\dagger(R_i-\hat{y}/2)$ with the hopping amplitude $-t_pp$.  
7. $c_{p_x,s}^\dagger(R_i-\hat{x}/2)$ and $c_{p_y,s}^\dagger(R_i+\hat{y}/2)$ with the hopping amplitude $-t_pp$.  
8. $c_{p_x,s}^\dagger(R_i-\hat{x}/2)$ and $c_{p_y,s}^\dagger(R_i-\hat{y}/2)$ with the hopping amplitude $-t_pp$.''',
    'kinetic_symbol': r'$H_0$',
    'You should ensure the hermiticity of the Hamiltonian|None': 'You should ensure the hermiticity of the Hamiltonian.',
    'def_var':r'''$s$ : spin index  
$\uparrow$ : spin up  
$\downarrow$ : spin down 
$\alpha$ : orbital index
$d$: $d$ orbital in Cu atom
$p_x$: $p_x$ orbital in O atom
$p_y$: $p_y$ orbital in O atom
$c_{\alpha,s}^\dagger(R_i)$ : annihilation operator for a particle with orbital $\alpha$ and spin $s$ at site $i$ 
$R_i$ : real space position of site $i$  
$\hat{x}$: unit vector along $x$ direction
$\hat{y}$: unit vector along $y$ direction
$t_{pd}$: hopping amplitude between $p_x$ and $d$ orbital, or $p_y$ and $d$ orbital
$t_{pp}$: hopping amplitude between $p_x$ and $p_y$ orbital''',
    'answer':    r'''$$
H_0 = -t_{pd}\sum_{R_i, s} ( c_{d,s}^\dagger(R_i) c_{p_x,s}(R_i+\hat{x}/2) + c_{p_x,s}^\dagger(R_i+\hat{x}/2) c_{d,s}(R_i) 
+ c_{d,s}^\dagger(R_i) c_{p_x,s}(R_i-\hat{x}/2) + c_{p_x,s}^\dagger(R_i-\hat{x}/2) c_{d,s}(R_i)
+ c_{d,s}^\dagger(R_i) c_{p_y,s}(R_i+\hat{y}/2) + c_{p_y,s}^\dagger(R_i+\hat{y}/2) c_{d,s}(R_i)
+ c_{d,s}^\dagger(R_i) c_{p_y,s}(R_i-\hat{y}/2) + c_{p_y,s}^\dagger(R_i-\hat{y}/2) c_{d,s}(R_i) )
-t_{pp}\sum_{R_i, s} ( c_{p_x,s}^\dagger(R_i+\hat{x}/2) c_{p_y,s}(R_i+\hat{y}/2) + c_{p_y,s}^\dagger(R_i+\hat{y}/2) c_{p_x,s}(R_i+\hat{x}/2) 
+ c_{p_x,s}^\dagger(R_i+\hat{x}/2) c_{p_y,s}(R_i-\hat{y}/2) + c_{p_y,s}^\dagger(R_i-\hat{y}/2) c_{p_x,s}(R_i+\hat{x}/2) 
+ c_{p_x,s}^\dagger(R_i-\hat{x}/2) c_{p_y,s}(R_i+\hat{y}/2) + c_{p_y,s}^\dagger(R_i+\hat{y}/2) c_{p_x,s}(R_i-\hat{x}/2) 
+ c_{p_x,s}^\dagger(R_i-\hat{x}/2) c_{p_y,s}(R_i-\hat{y}/2) + c_{p_y,s}^\dagger(R_i-\hat{y}/2) c_{p_x,s}(R_i-\hat{x}/2) )
$$''',
'source': {'1106.6060.tex':[(126,145)]}
}


In [12]:
# text=''
# for file, position in kwargs['source'].items():
#     with open(file,'r') as f:
#         f_list=list(f)
#         for left,right in position:
#             text+=''.join(f_list[left-1:right])
# print(text)

In [13]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to describe the kinetic term of Hamiltonian in CuO$_2$ in the real space in the second-quantized form.   
The degrees of freedom of the system are: spin index (spin up and spin down), orbital index ($d$ orbital in Cu atom, $p_x$ orbital in O atom, and $p_y$ orbital in O atom)     
The kinetic term is a tight-binding model composed of the following hopping process: 
1. $c_{d,s}^\dagger(R_i)$ and $c_{p_x,s}^\dagger(R_i+\hat{x}/2)$ with the hopping amplitude $-t_pd$.  
2. $c_{d,s}^\dagger(R_i)$ and $c_{p_x,s}^\dagger(R_i-\hat{x}/2)$ with the hopping amplitude $-t_pd$.  
3. $c_{d,s}^\dagger(R_i)$ and $c_{p_y,s}^\dagger(R_i+\hat{y}/2)$ with the hopping amplitude $-t_pd$.  
4. $c_{d,s}^\dagger(R_i)$ and $c_{p_y,s}^\dagger(R_i-\hat{y}/2)$ with the hopping amplitude $-t_pd$.  
5. $c_{p_x,s}^\dagger(R_i+\hat{x}/2)$ and $c_{p_y,s}^\dagger(R_i+\hat{y}/2)$ with the hopping amplitude $-t_pp$.  
6. $c_{p_x,s}^\dagger(R_i+\hat{x}/2)$ and $c_{p_y,s}^\dagger(R_i-\hat{y}/2)$ with 

In [14]:
# user_msg.append(generate_prompt(kwargs,prompt_dict=prompt_dict))

In [15]:
# messages=assembly_message(sys_msg,user_msg,AI_msg)

In [16]:
# rs = openai.ChatCompletion.create(messages=messages, **model_params)

# Markdown(rs['choices'][0]['message'].content)

In [17]:
# AI_msg.append(dict(rs['choices'][0]['message']))

In [18]:
# rs['choices'][0]['message'].content

In [19]:
kwargs_list.append(kwargs)

## Construct interaction Hamiltonian (real space, lattice version)

In [20]:
kwargs = {
    'task':'Construct interaction Hamiltonian (real space, lattice version)',
    'second_int_symbol':  r'$\hat{H}^{int}$',
    'kinetic_symbol': r'$H_0$',
    'site i and site j with the interaction strength': r'''1. onsite interaction between $n_{d,\uparrow}(R_i)$ and $n_{d,\downarrow}(R_i)$ with interaction strength $U_d$
2. onsite interaction between $n_{p_x,\uparrow}(R_i+\hat{x}/2)$ and $n_{p_x,\downarrow}(R_i+\hat{x}/2)$ with interaction strength $U_p/2$
3. onsite interaction between $n_{p_y,\uparrow}(R_i+\hat{y}/2)$ and $n_{p_y,\downarrow}(R_i+\hat{y}/2)$ with interaction strength $U_p/2$
4. Nearest neighbor interaction between $n_{d,s}(R_i)$ and $n_{p_x,s^{\prime}}(R_i+\hat{x}/2)$ with interaction strength $V_{pd}$
5. Nearest neighbor interaction between $n_{d,s}(R_i)$ and $n_{p_x,s^{\prime}}(R_i-\hat{x}/2)$ with interaction strength $V_{pd}$
6. Nearest neighbor interaction between $n_{d,s}(R_i)$ and $n_{p_y,s^{\prime}}(R_i+\hat{y}/2)$ with interaction strength $V_{pd}$
7. Nearest neighbor interaction between $n_{d,s}(R_i)$ and $n_{p_y,s^{\prime}}(R_i-\hat{y}/2)$ with interaction strength $V_{pd}$
8. Nearest neighbor interaction between $n_{p_x,s}(R_i+\hat{x}/2)$ and $n_{p_y,s^{\prime}}(R_i+\hat{y}/2)$ with interaction strength $V_{pp}$
9. Nearest neighbor interaction between $n_{p_x,s}(R_i+\hat{x}/2)$ and $n_{p_y,s^{\prime}}(R_i-\hat{y}/2)$ with interaction strength $V_{pp}$
10. Nearest neighbor interaction between $n_{p_x,s}(R_i-\hat{x}/2)$ and $n_{p_y,s^{\prime}}(R_i+\hat{y}/2)$ with interaction strength $V_{pp}$
11. Nearest neighbor interaction between $n_{p_x,s}(R_i-\hat{x}/2)$ and $n_{p_y,s^{\prime}}(R_i-\hat{y}/2)$ with interaction strength $V_{pp}$  
where density operator $n_{\alpha,s}(R_i) = c_{\alpha,s}^\dagger(R_i) c_{\alpha,s}(R_i)$. ''',
    'dof':r'spin index (spin up and spin down), orbital index ($d$ orbital in Cu atom, $p_x$ orbital in O atom, and $p_y$ orbital in O atom)',
    'density_symbol':r'$n_{\alpha,s}(R_i)$',

    'def_var': r'''$s^{\prime}$: spin index  
$n_{\alpha,s}(R_i)$: density of particle with orbital $\alpha$ and spin $s$ at site $R_i$  
$U_d$: onsite interaction strength of $d$ orbital  
$U_p$: onsite interaction strength of $p_x$ orbital or $p_y$ orbital  
$V_{pd}$: Nearest neighbor interaction strength between $p_x$ orbital and $d$ orbital or between $p_y$ orbital and $d$ orbital  
$V_{pp}$: Nearest neighbor interaction strength between $p_x$ orbital and $p_y$ orbital  ''',
    'answer':r'''$$
\hat{H}^{int} = U_d \sum_{R_i} n_{d,\uparrow}(R_i) n_{d,\downarrow}(R_i) + \frac{U_p}{2} \sum_{R_i} [n_{p_x,\uparrow}(R_i+\hat{x}/2) n_{p_x,\downarrow}(R_i+\hat{x}/2) + n_{p_y,\uparrow}(R_i+\hat{y}/2) n_{p_y,\downarrow}(R_i+\hat{y}/2)] \\ 
+ V_{pd} \sum_{R_i, s, s^{\prime}} [n_{d,s}(R_i) (n_{p_x,s^{\prime}}(R_i+\hat{x}/2) + n_{p_x,s^{\prime}}(R_i-\hat{x}/2) + n_{p_y,s^{\prime}}(R_i+\hat{y}/2) + n_{p_y,s^{\prime}}(R_i-\hat{y}/2)) ] \\
+ V_{pp} \sum_{R_i, s, s^{\prime}} [(n_{p_x,s}(R_i+\hat{x}/2) + n_{p_x,s}(R_i-\hat{x}/2)) (n_{p_y,s^{\prime}}(R_i+\hat{y}/2) + n_{p_y,s^{\prime}}(R_i-\hat{y}/2))]
$$''',
'source': {'1106.6060.tex':[(147,156)]}
}


In [21]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to construct the interaction part of the Hamiltonian, $\hat{H}^{int}$ in the real space in the second-quantized form.   
The interacting Hamiltonian has the same degrees of freedom as the kinetic Hamiltonian $H_0$.  
The interaction is a density-density interaction composed of the following process:
1. onsite interaction between $n_{d,\uparrow}(R_i)$ and $n_{d,\downarrow}(R_i)$ with interaction strength $U_d$
2. onsite interaction between $n_{p_x,\uparrow}(R_i+\hat{x}/2)$ and $n_{p_x,\downarrow}(R_i+\hat{x}/2)$ with interaction strength $U_p/2$
3. onsite interaction between $n_{p_y,\uparrow}(R_i+\hat{y}/2)$ and $n_{p_y,\downarrow}(R_i+\hat{y}/2)$ with interaction strength $U_p/2$
4. Nearest neighbor interaction between $n_{d,s}(R_i)$ and $n_{p_x,s^{\prime}}(R_i+\hat{x}/2)$ with interaction strength $V_{pd}$
5. Nearest neighbor interaction between $n_{d,s}(R_i)$ and $n_{p_x,s^{\prime}}(R_i-\hat{x}/2)$ with interaction strength $V_{pd}$
6. Nearest neighbor interact

In [22]:
kwargs_list.append(kwargs)

## Convert noninteracting Hamiltonian in real space to momentum space (lattice version)

In [23]:
kwargs = {
    'task':'Convert noninteracting Hamiltonian in real space to momentum space (lattice version)',
    'nonint_symbol': r'$H_0$',
    'real_creation_op': r'$c_{\alpha,s}^\dagger(R_i)$',
    'momentum_creation_op': r'$c_{\alpha,s}^\dagger(k)$',
    'def_FT': r'$c_{\alpha,s}^\dagger(k)= \frac{1}{\sqrt{N}} \sum_i c_{\alpha,s}^\dagger(R_i) e^{i k \cdot R_i}$',
    'real_var': '$i$',
    'Note that hopping have no position dependence now|None':r'Note that $t_{pp}$ and $t_{pd}$ have no position dependence now.',
    'expression_nonint': r'''$H_0 = -t_{pd}\sum_{R_i, s} ( c_{d,s}^\dagger(R_i) c_{p_x,s}(R_i+\hat{x}/2) + c_{p_x,s}^\dagger(R_i+\hat{x}/2) c_{d,s}(R_i) + c_{d,s}^\dagger(R_i) c_{p_x,s}(R_i-\hat{x}/2) + c_{p_x,s}^\dagger(R_i-\hat{x}/2) c_{d,s}(R_i)+ c_{d,s}^\dagger(R_i) c_{p_y,s}(R_i+\hat{y}/2) + c_{p_y,s}^\dagger(R_i+\hat{y}/2) c_{d,s}(R_i)+ c_{d,s}^\dagger(R_i) c_{p_y,s}(R_i-\hat{y}/2) + c_{p_y,s}^\dagger(R_i-\hat{y}/2) c_{d,s}(R_i) )-t_{pp}\sum_{R_i, s} ( c_{p_x,s}^\dagger(R_i+\hat{x}/2) c_{p_y,s}(R_i+\hat{y}/2) + c_{p_y,s}^\dagger(R_i+\hat{y}/2) c_{p_x,s}(R_i+\hat{x}/2) + c_{p_x,s}^\dagger(R_i+\hat{x}/2) c_{p_y,s}(R_i-\hat{y}/2) + c_{p_y,s}^\dagger(R_i-\hat{y}/2) c_{p_x,s}(R_i+\hat{x}/2) + c_{p_x,s}^\dagger(R_i-\hat{x}/2) c_{p_y,s}(R_i+\hat{y}/2) + c_{p_y,s}^\dagger(R_i+\hat{y}/2) c_{p_x,s}(R_i-\hat{x}/2) + c_{p_x,s}^\dagger(R_i-\hat{x}/2) c_{p_y,s}(R_i-\hat{y}/2) + c_{p_y,s}^\dagger(R_i-\hat{y}/2) c_{p_x,s}(R_i-\hat{x}/2) )$''',
    'def_var': r'''$c_{\alpha,s}(k)$ : electron annihilation operator at momentum $k$, spin $s$ and orbital $\alpha$
$c_{\alpha,s}^\dagger(k)$ : electron creation operator at momentum $k$, spin $s$ and orbital $\alpha$
$N$ : number of the unit cells in the real space  ''',
    'answer':r'''$$
H_0 = -t_{pd}\sum_{k, s} ( c_{d,s}^\dagger(k) c_{p_x,s}(k) e^{i k_x /2} + c_{p_x,s}^\dagger(k) c_{d,s}(k) e^{-i k_x /2}
+ c_{d,s}^\dagger(k) c_{p_x,s}(k) e^{-i k_x /2} + c_{p_x,s}^\dagger(k) c_{d,s}(k) e^{i k_x /2}
+ c_{d,s}^\dagger(k) c_{p_y,s}(k) e^{i k_y /2} + c_{p_y,s}^\dagger(k) c_{d,s}(k) e^{-i k_y /2}
+ c_{d,s}^\dagger(k) c_{p_y,s}(k) e^{-i k_y /2} + c_{p_y,s}^\dagger(k) c_{d,s}(k) e^{i k_y /2} )
-t_{pp}\sum_{k, s} ( c_{p_x,s}^\dagger(k) c_{p_y,s}(k) e^{i (k_x+k_y) /2} + c_{p_y,s}^\dagger(k) c_{p_x,s}(k) e^{-i (k_x+k_y) /2} 
+ c_{p_x,s}^\dagger(k) c_{p_y,s}(k) e^{i (k_x-k_y) /2} + c_{p_y,s}^\dagger(k) c_{p_x,s}(k) e^{-i (k_x-k_y) /2} 
+ c_{p_x,s}^\dagger(k) c_{p_y,s}(k) e^{i (-k_x+k_y) /2} + c_{p_y,s}^\dagger(k) c_{p_x,s}(k) e^{-i (-k_x+k_y) /2} 
+ c_{p_x,s}^\dagger(k) c_{p_y,s}(k) e^{i (-k_x-k_y) /2} + c_{p_y,s}^\dagger(k) c_{p_x,s}(k) e^{-i (-k_x-k_y) /2} )
$$''',
'source': {'1106.6060.tex':[(221,267)]}
}


In [24]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to convert the noninteracting Hamiltonian $H_0$ in the second quantized form from the basis in real space to the basis in momentum space. 
To do that, you should apply the Fourier transformation to $c_{\alpha,s}^\dagger(R_i)$ in the real space to the $c_{\alpha,s}^\dagger(k)$ in the momentum space, which is defined as $c_{\alpha,s}^\dagger(k)= \frac{1}{\sqrt{N}} \sum_i c_{\alpha,s}^\dagger(R_i) e^{i k \cdot R_i}$, where $i$ is integrated over all sites in the entire real space. You should follow the EXAMPLE below to apply the Fourier transformation. Note that $t_{pp}$ and $t_{pd}$ have no position dependence now.
You should recall that $H_0 = -t_{pd}\sum_{R_i, s} ( c_{d,s}^\dagger(R_i) c_{p_x,s}(R_i+\hat{x}/2) + c_{p_x,s}^\dagger(R_i+\hat{x}/2) c_{d,s}(R_i) + c_{d,s}^\dagger(R_i) c_{p_x,s}(R_i-\hat{x}/2) + c_{p_x,s}^\dagger(R_i-\hat{x}/2) c_{d,s}(R_i)+ c_{d,s}^\dagger(R_i) c_{p_y,s}(R_i+\hat{y}/2) + c_{p_y,s}^\dagger(R_i+\hat{y}/2) c_{d,s}(R_i)+ c_{d,s}^\dagger(R

In [25]:
kwargs_list.append(kwargs)

## Mathematical simplify: inner product expansion

In [26]:
kwargs = {
    'task':'Mathematical simplify: inner product expansion',
    'symbol': r'$H_0$',
    'expression_symbol': r'$H_0 = -t_{pd} \sum_{k,s}  (e^{i k \cdot \hat{x}/2} c_{d,s}^\dagger(k) c_{p_x,s}(k) + e^{-i k \cdot \hat{x}/2} c_{p_x,s}^\dagger(k) c_{d,s}(k)+ e^{-i k \cdot \hat{x}/2} c_{d,s}^\dagger(k) c_{p_x,s}(k) + e^{i k \cdot \hat{x}/2} c_{p_x,s}^\dagger(k) c_{d,s}(k) + e^{i k \cdot \hat{y}/2} c_{d,s}^\dagger(k) c_{p_y,s}(k) + e^{-i k \cdot \hat{y}/2} c_{p_y,s}^\dagger(k) c_{d,s}(k)+ e^{-i k \cdot \hat{y}/2} c_{d,s}^\dagger(k) c_{p_y,s}(k) + e^{i k \cdot \hat{y}/2} c_{p_y,s}^\dagger(k) c_{d,s}(k)) - t_{pp} \sum_{k,s}  (e^{i k \cdot (\hat{x}+\hat{y})/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i k \cdot (\hat{x}+\hat{y})/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k) + e^{i k \cdot (\hat{x}-\hat{y})/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i k \cdot (\hat{x}-\hat{y})/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k)+ e^{i k \cdot (-\hat{x}+\hat{y})/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i k \cdot (-\hat{x}+\hat{y})/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k)+ e^{i k \cdot (-\hat{x}-\hat{y})/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i k \cdot (-\hat{x}-\hat{y})/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k))$',
    'answer':r'''$H_0 = -t_{pd} \sum_{k,s}  (e^{i k_x/2} c_{d,s}^\dagger(k) c_{p_x,s}(k) + e^{-i k_x/2} c_{p_x,s}^\dagger(k) c_{d,s}(k)+ e^{-i k_x/2} c_{d,s}^\dagger(k) c_{p_x,s}(k) + e^{i k_x/2} c_{p_x,s}^\dagger(k) c_{d,s}(k) + e^{i k_y/2} c_{d,s}^\dagger(k) c_{p_y,s}(k) + e^{-i k_y/2} c_{p_y,s}^\dagger(k) c_{d,s}(k)+ e^{-i k_y/2} c_{d,s}^\dagger(k) c_{p_y,s}(k) + e^{i k_y/2} c_{p_y,s}^\dagger(k) c_{d,s}(k))- t_{pp} \sum_{k,s}  (e^{i (k_x+k_y)/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i (k_x+k_y)/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k) + e^{i (k_x-k_y)/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i (k_x-k_y)/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k)+ e^{i (-k_x+k_y)/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i (-k_x+k_y)/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k)+ e^{i (-k_x-k_y)/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i (-k_x-k_y)/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k))$''',
'source': {}
}


In [27]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify $H_0$ by using $k\cdot \hat{x}=k_x$ and $k\cdot \hat{y}=k_y$ only.  
You should recall that $H_0 = -t_{pd} \sum_{k,s}  (e^{i k \cdot \hat{x}/2} c_{d,s}^\dagger(k) c_{p_x,s}(k) + e^{-i k \cdot \hat{x}/2} c_{p_x,s}^\dagger(k) c_{d,s}(k)+ e^{-i k \cdot \hat{x}/2} c_{d,s}^\dagger(k) c_{p_x,s}(k) + e^{i k \cdot \hat{x}/2} c_{p_x,s}^\dagger(k) c_{d,s}(k) + e^{i k \cdot \hat{y}/2} c_{d,s}^\dagger(k) c_{p_y,s}(k) + e^{-i k \cdot \hat{y}/2} c_{p_y,s}^\dagger(k) c_{d,s}(k)+ e^{-i k \cdot \hat{y}/2} c_{d,s}^\dagger(k) c_{p_y,s}(k) + e^{i k \cdot \hat{y}/2} c_{p_y,s}^\dagger(k) c_{d,s}(k)) - t_{pp} \sum_{k,s}  (e^{i k \cdot (\hat{x}+\hat{y})/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i k \cdot (\hat{x}+\hat{y})/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k) + e^{i k \cdot (\hat{x}-\hat{y})/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i k \cdot (\hat{x}-\hat{y})/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k)+ e^{i k \cdot (-\hat{x}+\hat{y})/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i 

In [28]:
kwargs_list.append(kwargs)

## Mathematical simplify: Euler's formula

In [29]:
kwargs = {
    'task':"Mathematical simplify: Euler's formula",
    'symbol': r'$H_0$',
    'expression_symbol': r'$H_0 = -t_{pd} \sum_{k,s}  (e^{i k_x/2} c_{d,s}^\dagger(k) c_{p_x,s}(k) + e^{-i k_x/2} c_{p_x,s}^\dagger(k) c_{d,s}(k)+ e^{-i k_x/2} c_{d,s}^\dagger(k) c_{p_x,s}(k) + e^{i k_x/2} c_{p_x,s}^\dagger(k) c_{d,s}(k) + e^{i k_y/2} c_{d,s}^\dagger(k) c_{p_y,s}(k) + e^{-i k_y/2} c_{p_y,s}^\dagger(k) c_{d,s}(k)+ e^{-i k_y/2} c_{d,s}^\dagger(k) c_{p_y,s}(k) + e^{i k_y/2} c_{p_y,s}^\dagger(k) c_{d,s}(k))- t_{pp} \sum_{k,s}  (e^{i (k_x+k_y)/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i (k_x+k_y)/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k) + e^{i (k_x-k_y)/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i (k_x-k_y)/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k)+ e^{i (-k_x+k_y)/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i (-k_x+k_y)/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k)+ e^{i (-k_x-k_y)/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i (-k_x-k_y)/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k))$',
    'answer':r'''$H_0 = -2 t_{pd}\sum_{k, s} ( c_{d,s}^\dagger(k) c_{p_x,s}(k) \cos(k_x /2) + c_{p_x,s}^\dagger(k) c_{d,s}(k) \cos(k_x /2) + c_{d,s}^\dagger(k) c_{p_y,s}(k) \cos(k_y /2) + c_{p_y,s}^\dagger(k) c_{d,s}(k) \cos(k_y /2) )-2 t_{pp}\sum_{k, s} ( c_{p_x,s}^\dagger(k) c_{p_y,s}(k) \cos((k_x+k_y) /2) + c_{p_y,s}^\dagger(k) c_{p_x,s}(k) \cos((k_x+k_y) /2)+ c_{p_x,s}^\dagger(k) c_{p_y,s}(k) \cos((k_x-k_y) /2) + c_{p_y,s}^\dagger(k) c_{p_x,s}(k) \cos((k_x-k_y) /2) )$''',
'source': {}
}


In [30]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify $H_0$ by converting the exponential to the trigonometrical functions using Euler's formula only. You should also apply basic trigonometrical function simplification afterwards if possible. 
You should recall that $H_0 = -t_{pd} \sum_{k,s}  (e^{i k_x/2} c_{d,s}^\dagger(k) c_{p_x,s}(k) + e^{-i k_x/2} c_{p_x,s}^\dagger(k) c_{d,s}(k)+ e^{-i k_x/2} c_{d,s}^\dagger(k) c_{p_x,s}(k) + e^{i k_x/2} c_{p_x,s}^\dagger(k) c_{d,s}(k) + e^{i k_y/2} c_{d,s}^\dagger(k) c_{p_y,s}(k) + e^{-i k_y/2} c_{p_y,s}^\dagger(k) c_{d,s}(k)+ e^{-i k_y/2} c_{d,s}^\dagger(k) c_{p_y,s}(k) + e^{i k_y/2} c_{p_y,s}^\dagger(k) c_{d,s}(k))- t_{pp} \sum_{k,s}  (e^{i (k_x+k_y)/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i (k_x+k_y)/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k) + e^{i (k_x-k_y)/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i (k_x-k_y)/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k)+ e^{i (-k_x+k_y)/2} c_{p_x,s}^\dagger(k) c_{p_y,s}(k) + e^{-i (-k_x+k_y)/2} c_{p_y,s}^\dagger(k) c_{p_x,s}(k)+ e^{i (-

In [31]:
kwargs_list.append(kwargs)

## Mathematical simplify: prosthaphaeresis

In [32]:
kwargs = {
    'task':"Mathematical simplify: prosthaphaeresis",
    'symbol': r'$H_0$',
    'expression_symbol': r'$H_0 = -2 t_{pd}\sum_{k, s} ( c_{d,s}^\dagger(k) c_{p_x,s}(k) \cos(k_x /2) + c_{p_x,s}^\dagger(k) c_{d,s}(k) \cos(k_x /2) + c_{d,s}^\dagger(k) c_{p_y,s}(k) \cos(k_y /2) + c_{p_y,s}^\dagger(k) c_{d,s}(k) \cos(k_y /2) )-2 t_{pp}\sum_{k, s} ( c_{p_x,s}^\dagger(k) c_{p_y,s}(k) \cos((k_x+k_y) /2) + c_{p_y,s}^\dagger(k) c_{p_x,s}(k) \cos((k_x+k_y) /2)+ c_{p_x,s}^\dagger(k) c_{p_y,s}(k) \cos((k_x-k_y) /2) + c_{p_y,s}^\dagger(k) c_{p_x,s}(k) \cos((k_x-k_y) /2) )$',
    'answer':r'''$$
H_0 = -2 t_{pd}\sum_{k, s} ( c_{d,s}^\dagger(k) c_{p_x,s}(k) \cos(k_x /2) + c_{p_x,s}^\dagger(k) c_{d,s}(k) \cos(k_x /2) + c_{d,s}^\dagger(k) c_{p_y,s}(k) \cos(k_y /2) + c_{p_y,s}^\dagger(k) c_{d,s}(k) \cos(k_y /2) )-4 t_{pp}\sum_{k, s} ( c_{p_x,s}^\dagger(k) c_{p_y,s}(k) \cos(k_x/2) \cos(k_y/2) + c_{p_y,s}^\dagger(k) c_{p_x,s}(k) \cos(k_x/2) \cos(k_y/2))
$$''',
'source': {}
}


In [33]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify $H_0$ by reducing trigonometrical functions using prosthaphaeresis.  
You should recall that $H_0 = -2 t_{pd}\sum_{k, s} ( c_{d,s}^\dagger(k) c_{p_x,s}(k) \cos(k_x /2) + c_{p_x,s}^\dagger(k) c_{d,s}(k) \cos(k_x /2) + c_{d,s}^\dagger(k) c_{p_y,s}(k) \cos(k_y /2) + c_{p_y,s}^\dagger(k) c_{d,s}(k) \cos(k_y /2) )-2 t_{pp}\sum_{k, s} ( c_{p_x,s}^\dagger(k) c_{p_y,s}(k) \cos((k_x+k_y) /2) + c_{p_y,s}^\dagger(k) c_{p_x,s}(k) \cos((k_x+k_y) /2)+ c_{p_x,s}^\dagger(k) c_{p_y,s}(k) \cos((k_x-k_y) /2) + c_{p_y,s}^\dagger(k) c_{p_x,s}(k) \cos((k_x-k_y) /2) )$.  
Return the simplified $H_0$.


In [34]:
kwargs_list.append(kwargs)

## Convert interacting Hamiltonian in real space to momentum space (lattice version) [first term]

In [35]:
kwargs = {
    'task':'Convert interacting Hamiltonian in real space to momentum space (lattice version)',
    'second_int_symbol':  r'the first term in $\hat{H}^{int}$',
    'single-particle|second-quantized': r'second-quantized',
    'real_creation_op': r'$c_{\alpha,s}^\dagger(R_i)$',
    'momentum_creation_op': r'$c_{\alpha,s}^\dagger(k)$',
    'def_FT': r'$c_{\alpha,s}^\dagger(k)= \frac{1}{\sqrt{N}} \sum_i c_{\alpha,s}^\dagger(R_i) e^{i k \cdot R_i}$',
    'real_var': '$i$',
    'momentum_var': '$k$',
    'Note that interaction have no position dependence now|None':'Note that interaction have no position dependence',
    'expression_int':r'''$U_d \sum_{R_i} n_{d,\uparrow}(R_i) n_{d,\downarrow}(R_i)$''',
    'momentum_creation_op': r'$c_{\alpha,s}^\dagger(k)$',
    'answer':r'''$\frac{U_d}{N}\sum_{k_1,k_2,k_3,k_4} c_{d,\uparrow}^\dagger(k_1)c_{d,\uparrow}(k_2)c_{d,\downarrow}^\dagger(k_3)c_{d,\downarrow}(k_4) \sum_{G} \delta(k_1-k_2+k_3-k_4,G)$''',
'source': {'1106.6060.tex':[(221,267)]}
}


In [36]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to convert the interacting Hamiltonian, the first term in $\hat{H}^{int}$, in the second-quantized form the basis in real space to the basis in momentum space.
To do that, you should apply the Fourier transformation to $c_{\alpha,s}^\dagger(R_i)$ in the real space to the $c_{\alpha,s}^\dagger(k)$ in the momentum space, which is defined as $c_{\alpha,s}^\dagger(k)= \frac{1}{\sqrt{N}} \sum_i c_{\alpha,s}^\dagger(R_i) e^{i k \cdot R_i}$, where $i$ is integrated over all sites in the entire real space, and $k$ is defined within the first Brillouin zone. You should follow the EXAMPLE below to apply the Fourier transformation. Note that interaction have no position dependence
You should recall that the first term in $\hat{H}^{int}$ is $U_d \sum_{R_i} n_{d,\uparrow}(R_i) n_{d,\downarrow}(R_i)$.
Express the first term in $\hat{H}^{int}$ in terms of $c_{\alpha,s}^\dagger(k)$. Simplify any summation index if possible.  

===  
EXAMPLE:  
Write an interacting Hamiltonian $\

In [37]:
kwargs_list.append(kwargs)

## Convert interacting Hamiltonian in real space to momentum space (lattice version) [second term]

In [38]:
kwargs = {
    'task':'Convert interacting Hamiltonian in real space to momentum space (lattice version)',
    'second_int_symbol':  r'the second term in $\hat{H}^{int}$',
    'single-particle|second-quantized': r'second-quantized',
    'real_creation_op': r'$c_{\alpha,s}^\dagger(R_i)$',
    'momentum_creation_op': r'$c_{\alpha,s}^\dagger(k)$',
    'def_FT': r'$c_{\alpha,s}^\dagger(k)= \frac{1}{\sqrt{N}} \sum_i c_{\alpha,s}^\dagger(R_i) e^{i k \cdot R_i}$',
    'real_var': '$i$',
    'momentum_var': '$k$',
    'Note that interaction have no position dependence now|None':'Note that interaction have no position dependence',
    'expression_int':r'''$\frac{U_p}{2} \sum_{R_i} [n_{p_x,\uparrow}(R_i+\hat{x}/2) n_{p_x,\downarrow}(R_i+\hat{x}/2) + n_{p_y,\uparrow}(R_i+\hat{y}/2) n_{p_y,\downarrow}(R_i+\hat{y}/2)]$''',
    'momentum_creation_op': r'$c_{\alpha,s}^\dagger(k)$',
    'answer':r'''$\hat{H}_{int}^{(2)} = \frac{U_p}{2N} \sum_{k_1,k_2,k_3,k_4} \{c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \sum_{G} \delta(k_1-k_2+k_3-k_4,G) + c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4) \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \}.$''',
'source': {'1106.6060.tex':[(221,267)]}
}


In [39]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to convert the interacting Hamiltonian, the second term in $\hat{H}^{int}$, in the second-quantized form the basis in real space to the basis in momentum space.
To do that, you should apply the Fourier transformation to $c_{\alpha,s}^\dagger(R_i)$ in the real space to the $c_{\alpha,s}^\dagger(k)$ in the momentum space, which is defined as $c_{\alpha,s}^\dagger(k)= \frac{1}{\sqrt{N}} \sum_i c_{\alpha,s}^\dagger(R_i) e^{i k \cdot R_i}$, where $i$ is integrated over all sites in the entire real space, and $k$ is defined within the first Brillouin zone. You should follow the EXAMPLE below to apply the Fourier transformation. Note that interaction have no position dependence
You should recall that the second term in $\hat{H}^{int}$ is $\frac{U_p}{2} \sum_{R_i} [n_{p_x,\uparrow}(R_i+\hat{x}/2) n_{p_x,\downarrow}(R_i+\hat{x}/2) + n_{p_y,\uparrow}(R_i+\hat{y}/2) n_{p_y,\downarrow}(R_i+\hat{y}/2)]$.
Express the second term in $\hat{H}^{int}$ in terms of $c_{\alpha,s}^\da

In [40]:
kwargs_list.append(kwargs)

## Mathematical simplify: Expand the product using Associative property

In [41]:
kwargs = {
    'task':"Mathematical simplify: Expand the product using Associative property",
    'symbol': r'the third term with $V_{pd}$ in $\hat{H}^{int}$',
    'expression_symbol': r'''the third term in $\hat{H}^{int}$ is $V_{pd} \sum_{R_i, s, s^{\prime}} [n_{d,s}(R_i) (n_{p_x,s^{\prime}}(R_i+\hat{x}/2) + n_{p_x,s^{\prime}}(R_i-\hat{x}/2) + n_{p_y,s^{\prime}}(R_i+\hat{y}/2) + n_{p_y,s^{\prime}}(R_i-\hat{y}/2)) ]$''',

    'answer':r'''$H_0 = -2 t_{pd}\sum_{k, s} ( c_{d,s}^\dagger(k) c_{p_x,s}(k) \cos(k_x /2) + c_{p_x,s}^\dagger(k) c_{d,s}(k) \cos(k_x /2) + c_{d,s}^\dagger(k) c_{p_y,s}(k) \cos(k_y /2) + c_{p_y,s}^\dagger(k) c_{d,s}(k) \cos(k_y /2) )-2 t_{pp}\sum_{k, s} ( c_{p_x,s}^\dagger(k) c_{p_y,s}(k) \cos((k_x+k_y) /2) + c_{p_y,s}^\dagger(k) c_{p_x,s}(k) \cos((k_x+k_y) /2)+ c_{p_x,s}^\dagger(k) c_{p_y,s}(k) \cos((k_x-k_y) /2) + c_{p_y,s}^\dagger(k) c_{p_x,s}(k) \cos((k_x-k_y) /2) )$''',
'source': {}
}


In [42]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to expand the product in the third term with $V_{pd}$ in $\hat{H}^{int}$.  
You should recall that the third term in $\hat{H}^{int}$ is $V_{pd} \sum_{R_i, s, s^{\prime}} [n_{d,s}(R_i) (n_{p_x,s^{\prime}}(R_i+\hat{x}/2) + n_{p_x,s^{\prime}}(R_i-\hat{x}/2) + n_{p_y,s^{\prime}}(R_i+\hat{y}/2) + n_{p_y,s^{\prime}}(R_i-\hat{y}/2)) ]$.   
Express the third term with $V_{pd}$ in $\hat{H}^{int}$ in the expanded form.


In [43]:
kwargs_list.append(kwargs)

## Convert interacting Hamiltonian in real space to momentum space (lattice version) [third term]

In [44]:
kwargs = {
    'task':'Convert interacting Hamiltonian in real space to momentum space (lattice version)',
    'second_int_symbol':  r'the third term in $\hat{H}^{int}$',
    'single-particle|second-quantized': r'second-quantized',
    'real_creation_op': r'$c_{\alpha,s}^\dagger(R_i)$',
    'momentum_creation_op': r'$c_{\alpha,s}^\dagger(k)$',
    'def_FT': r'$c_{\alpha,s}^\dagger(k)= \frac{1}{\sqrt{N}} \sum_i c_{\alpha,s}^\dagger(R_i) e^{i k \cdot R_i}$',
    'real_var': '$i$',
    'momentum_var': '$k$',
    'Note that interaction have no position dependence now|None':'Note that interaction have no position dependence',
    'expression_int':r'''$$ V_{pd} \sum_{R_i, s, s^{\prime}} [n_{d,s}(R_i)n_{p_x,s^{\prime}}(R_i+\hat{x}/2) + n_{d,s}(R_i)n_{p_x,s^{\prime}}(R_i-\hat{x}/2) + n_{d,s}(R_i)n_{p_y,s^{\prime}}(R_i+\hat{y}/2) + n_{d,s}(R_i)n_{p_y,s^{\prime}}(R_i-\hat{y}/2)] $$''',
    'momentum_creation_op': r'$c_{\alpha,s}^\dagger(k)$',
    'answer':r'''$$
\hat{H}_{pd}^{int} = 2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \Big[ c_{d,s}^\dagger(k_1)c_{d,s}(k_2)c_{p_x,s^{\prime}}^\dagger(k_3)c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)+ c_{d,s}^\dagger(k_1)c_{d,s}(k_2)c_{p_y,s^{\prime}}^\dagger(k_3)c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right) \Big] \sum_{G} \delta(k_1-k_2+k_3-k_4,G)
$$''',
'source': {'1106.6060.tex':[(221,267)]}
}


In [45]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to convert the interacting Hamiltonian, the third term in $\hat{H}^{int}$, in the second-quantized form the basis in real space to the basis in momentum space.
To do that, you should apply the Fourier transformation to $c_{\alpha,s}^\dagger(R_i)$ in the real space to the $c_{\alpha,s}^\dagger(k)$ in the momentum space, which is defined as $c_{\alpha,s}^\dagger(k)= \frac{1}{\sqrt{N}} \sum_i c_{\alpha,s}^\dagger(R_i) e^{i k \cdot R_i}$, where $i$ is integrated over all sites in the entire real space, and $k$ is defined within the first Brillouin zone. You should follow the EXAMPLE below to apply the Fourier transformation. Note that interaction have no position dependence
You should recall that the third term in $\hat{H}^{int}$ is $$ V_{pd} \sum_{R_i, s, s^{\prime}} [n_{d,s}(R_i)n_{p_x,s^{\prime}}(R_i+\hat{x}/2) + n_{d,s}(R_i)n_{p_x,s^{\prime}}(R_i-\hat{x}/2) + n_{d,s}(R_i)n_{p_y,s^{\prime}}(R_i+\hat{y}/2) + n_{d,s}(R_i)n_{p_y,s^{\prime}}(R_i-\hat{y}/2)] $$.
Expres

In [46]:
kwargs_list.append(kwargs)

## Mathematical simplify: Combine using Associative property

In [47]:
kwargs = {
    'task':"Mathematical simplify: Combine using Associative property",
    'symbol': r'the fourth term with $V_{pp}$ in $\hat{H}^{int}$',
    'var': r'$\sigma_x$ before $\hat{x}/2$ and $\sigma_y$ before $\hat{y}/2$',
    'values': r'$\pm 1$',
    'num_of_terms': 'four',
    'expressoin_symbol':r'''the fourth term in $\hat{H}^{int}$ is $V_{pp} \sum_{R_i, s, s^{\prime}} [(n_{p_x,s}(R_i+\hat{x}/2) + n_{p_x,s}(R_i-\hat{x}/2)) (n_{p_y,s^{\prime}}(R_i+\hat{y}/2) + n_{p_y,s^{\prime}}(R_i-\hat{y}/2))]$''',

    'answer':r'''$V_{pp} \sum_{R_i, s, s^{\prime}} \sum_{\sigma_x, \sigma_y} n_{p_x,s}(R_i+\sigma_x\hat{x}/2) n_{p_y,s^{\prime}}(R_i+\sigma_y\hat{y}/2)$''',
'source': {}
}


In [48]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to expand the product in the fourth term with $V_{pp}$ in $\hat{H}^{int}$.  
After expansion, you can introduce a sign variable $\sigma_x$ before $\hat{x}/2$ and $\sigma_y$ before $\hat{y}/2$, which take values of $\pm 1$ to combine the four term in to one term.
You should recall that the fourth term in $\hat{H}^{int}$ is $V_{pp} \sum_{R_i, s, s^{\prime}} [(n_{p_x,s}(R_i+\hat{x}/2) + n_{p_x,s}(R_i-\hat{x}/2)) (n_{p_y,s^{\prime}}(R_i+\hat{y}/2) + n_{p_y,s^{\prime}}(R_i-\hat{y}/2))]$.
Express the the fourth term with $V_{pp}$ in $\hat{H}^{int}$ in the expanded form.


In [49]:
kwargs_list.append(kwargs)

## Convert interacting Hamiltonian in real space to momentum space (lattice version) [fourth term]

In [50]:
kwargs = {
    'task':'Convert interacting Hamiltonian in real space to momentum space (lattice version)',
    'second_int_symbol':  r'the fourth term in $\hat{H}^{int}$',
    'single-particle|second-quantized': r'second-quantized',
    'real_creation_op': r'$c_{\alpha,s}^\dagger(R_i)$',
    'momentum_creation_op': r'$c_{\alpha,s}^\dagger(k)$',
    'def_FT': r'$c_{\alpha,s}^\dagger(k)= \frac{1}{\sqrt{N}} \sum_i c_{\alpha,s}^\dagger(R_i) e^{i k \cdot R_i}$',
    'real_var': '$i$',
    'momentum_var': '$k$',
    'Note that interaction have no position dependence now|None':'Note that interaction have no position dependence',
    'expression_int':r'''$V_{pp} \sum_{R_i, s, s^{\prime}} \sum_{\sigma_x, \sigma_y} n_{p_x,s}(R_i+\sigma_x\hat{x}/2) n_{p_y,s^{\prime}}(R_i+\sigma_y\hat{y}/2)$, where $\sigma_x$ and $\sigma_y$ both take value of $\pm 1$''',
    'momentum_creation_op': r'$c_{\alpha,s}^\dagger(k)$',
    'answer':r'''$$
\hat{H}_{pd}^{int} = 2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \Big[ c_{d,s}^\dagger(k_1)c_{d,s}(k_2)c_{p_x,s^{\prime}}^\dagger(k_3)c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)+ c_{d,s}^\dagger(k_1)c_{d,s}(k_2)c_{p_y,s^{\prime}}^\dagger(k_3)c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right) \Big] \sum_{G} \delta(k_1-k_2+k_3-k_4,G)
$$''',
'source': {'1106.6060.tex':[(221,267)]}
}


In [51]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to convert the interacting Hamiltonian, the fourth term in $\hat{H}^{int}$, in the second-quantized form the basis in real space to the basis in momentum space.
To do that, you should apply the Fourier transformation to $c_{\alpha,s}^\dagger(R_i)$ in the real space to the $c_{\alpha,s}^\dagger(k)$ in the momentum space, which is defined as $c_{\alpha,s}^\dagger(k)= \frac{1}{\sqrt{N}} \sum_i c_{\alpha,s}^\dagger(R_i) e^{i k \cdot R_i}$, where $i$ is integrated over all sites in the entire real space, and $k$ is defined within the first Brillouin zone. You should follow the EXAMPLE below to apply the Fourier transformation. Note that interaction have no position dependence
You should recall that the fourth term in $\hat{H}^{int}$ is $V_{pp} \sum_{R_i, s, s^{\prime}} \sum_{\sigma_x, \sigma_y} n_{p_x,s}(R_i+\sigma_x\hat{x}/2) n_{p_y,s^{\prime}}(R_i+\sigma_y\hat{y}/2)$, where $\sigma_x$ and $\sigma_y$ both take value of $\pm 1$.
Express the fourth term in $\hat{H}^{in

In [52]:
kwargs_list.append(kwargs)

## Wick's theorem [first term]

In [53]:
kwargs = {
    'task':"Wick's theorem",
    'second_int_symbol':  r'the first term in $\hat{H}^{int}$',
    'HF_symbol': r'the first term in $\hat{H}^{int,HF}$',
    'expression_int': r'''the first term is $\frac{U_d}{N}\sum_{k_1,k_2,k_3,k_4} c_{d,\uparrow}^\dagger(k_1)c_{d,\uparrow}(k_2)c_{d,\downarrow}^\dagger(k_3)c_{d,\downarrow}(k_4) \sum_{G} \delta(k_1-k_2+k_3-k_4,G)$''',
    'def_var': r'''$\langle c_{\alpha,s}^\dagger(k_1) c_{\alpha^{\prime},s^{\prime}}(k_2) \rangle$ : expectation of $c_{\alpha,s}^\dagger(k_1) c_{\alpha^{\prime},s^{\prime}}(k_2)$ over the many-body ground states   
$\hat{H}^{int,HF}$ : Interaction term after Hartree-Fock approximation  ''',
    'answer':r'''$\hat{H}^{int,HF,U_d} = \frac{U_d}{N}\sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \big[ \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4)
+ \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) \rangle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2)
- \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \rangle \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) \rangle
- \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\downarrow}(k_4) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\uparrow}(k_2)
- \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\uparrow}(k_2) \rangle c_{d,\uparrow}^\dagger(k_1) c_{d,\downarrow}(k_4)
+ \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\downarrow}(k_4) \rangle \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\uparrow}(k_2) \rangle \big]$''',
'source': {'1106.6060.tex':[(221,267)]}
}


In [54]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to perform a Hartree-Fock approximation to expand the interaction term, the first term in $\hat{H}^{int}$.  
You should use Wick's theorem to expand the four-fermion term in the first term in $\hat{H}^{int}$ into quadratic terms. You should strictly follow the EXAMPLE below to expand using Wick's theorem, select the correct EXAMPLE by noticing the order of four term product with and without ${}^\dagger$, and be extremely cautious about the order of the index and sign before each term.  
You should only preserve the normal terms. Here, the normal terms mean the product of a creation operator and an annihilation operator.  
You should recall that the first term is $\frac{U_d}{N}\sum_{k_1,k_2,k_3,k_4} c_{d,\uparrow}^\dagger(k_1)c_{d,\uparrow}(k_2)c_{d,\downarrow}^\dagger(k_3)c_{d,\downarrow}(k_4) \sum_{G} \delta(k_1-k_2+k_3-k_4,G)$.  
Return the expanded interaction term after Hartree-Fock approximation as the first term in $\hat{H}^{int,HF}$.

Use the following con

In [55]:
kwargs_list.append(kwargs)

## Wick's theorem [second term]

In [56]:
kwargs = {
    'task':"Wick's theorem",
    'second_int_symbol':  r'the second term in $\hat{H}^{int}$',
    'HF_symbol': r'the second term in $\hat{H}^{int,HF}$',
    'expression_int': r'''the second term is $\frac{U_p}{2N} \sum_{k_1,k_2,k_3,k_4} \{c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \sum_{G} \delta(k_1-k_2+k_3-k_4,G) + c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4) \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \}$''',
    'def_var': r'''$\langle c_{\alpha,s}^\dagger(k_1) c_{\alpha^{\prime},s^{\prime}}(k_2) \rangle$ : expectation of $c_{\alpha,s}^\dagger(k_1) c_{\alpha^{\prime},s^{\prime}}(k_2)$ over the many-body ground states   
$\hat{H}^{int,HF}$ : Interaction term after Hartree-Fock approximation  ''',
    'answer':r'''$\frac{U_p}{2N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big\{ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4)
+ \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \rangle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2)
- \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \rangle
- \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\downarrow}(k_4) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\uparrow}(k_2)
- \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\downarrow}(k_4)
+ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\downarrow}(k_4) \rangle \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\uparrow}(k_2) \rangle + \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) \rangle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4)
+ \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4) \rangle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2)
- \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) \rangle \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4) \rangle
- \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\downarrow}(k_4) \rangle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\uparrow}(k_2)
- \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\uparrow}(k_2) \rangle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\downarrow}(k_4)
+ \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\downarrow}(k_4) \rangle \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\uparrow}(k_2) \rangle \Big\}$''',
'source': {'1106.6060.tex':[(221,267)]}

}


In [57]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to perform a Hartree-Fock approximation to expand the interaction term, the second term in $\hat{H}^{int}$.  
You should use Wick's theorem to expand the four-fermion term in the second term in $\hat{H}^{int}$ into quadratic terms. You should strictly follow the EXAMPLE below to expand using Wick's theorem, select the correct EXAMPLE by noticing the order of four term product with and without ${}^\dagger$, and be extremely cautious about the order of the index and sign before each term.  
You should only preserve the normal terms. Here, the normal terms mean the product of a creation operator and an annihilation operator.  
You should recall that the second term is $\frac{U_p}{2N} \sum_{k_1,k_2,k_3,k_4} \{c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \sum_{G} \delta(k_1-k_2+k_3-k_4,G) + c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4) \sum_{G} 

In [58]:
kwargs_list.append(kwargs)

## Wick's theorem [third term]

In [59]:
kwargs = {
    'task':"Wick's theorem",
    'second_int_symbol':  r'the third term in $\hat{H}^{int}$',
    'HF_symbol': r'the third term in $\hat{H}^{int,HF}$',
    'expression_int': r'''the third term is $2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \Big[ c_{d,s}^\dagger(k_1)c_{d,s}(k_2)c_{p_x,s^{\prime}}^\dagger(k_3)c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)+ c_{d,s}^\dagger(k_1)c_{d,s}(k_2)c_{p_y,s^{\prime}}^\dagger(k_3)c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right) \Big] \sum_{G} \delta(k_1-k_2+k_3-k_4,G)$''',
    'def_var': r'''$\langle c_{\alpha,s}^\dagger(k_1) c_{\alpha^{\prime},s^{\prime}}(k_2) \rangle$ : expectation of $c_{\alpha,s}^\dagger(k_1) c_{\alpha^{\prime},s^{\prime}}(k_2)$ over the many-body ground states   
$\hat{H}^{int,HF}$ : Interaction term after Hartree-Fock approximation  ''',
    'answer':r'''$2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[
\langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right) 
+ \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)
- \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \rangle \cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)\\
- \langle c_{d,s}^\dagger(k_1) c_{p_x,s^{\prime}}(k_4) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)
- \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle c_{d,s}^\dagger(k_1) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)
+ \langle c_{d,s}^\dagger(k_1) c_{p_x,s^{\prime}}(k_4) \rangle \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle \cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)\\
+ \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)
+ \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)
- \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle \cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)\\
- \langle c_{d,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)
- \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle c_{d,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)
+ \langle c_{d,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle \cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)
\Big]$''',
'source': {'1106.6060.tex':[(221,267)]}

}


In [60]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to perform a Hartree-Fock approximation to expand the interaction term, the third term in $\hat{H}^{int}$.  
You should use Wick's theorem to expand the four-fermion term in the third term in $\hat{H}^{int}$ into quadratic terms. You should strictly follow the EXAMPLE below to expand using Wick's theorem, select the correct EXAMPLE by noticing the order of four term product with and without ${}^\dagger$, and be extremely cautious about the order of the index and sign before each term.  
You should only preserve the normal terms. Here, the normal terms mean the product of a creation operator and an annihilation operator.  
You should recall that the third term is $2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \Big[ c_{d,s}^\dagger(k_1)c_{d,s}(k_2)c_{p_x,s^{\prime}}^\dagger(k_3)c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)+ c_{d,s}^\dagger(k_1)c_{d,s}(k_2)c_{p_y,s^{\prime}}^\dagger(k_3)c_{p_y,s^{\prime}}(k_4)\cos\left(\fra

In [61]:
kwargs_list.append(kwargs)

## Wick's theorem [fourth term]

In [62]:
kwargs = {
    'task':"Wick's theorem",
    'second_int_symbol':  r'the fourth term in $\hat{H}^{int}$',
    'HF_symbol': r'the fourth term in $\hat{H}^{int,HF}$',
    'expression_int': r'''the fourth term is $\frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_2 - k_1 + k_4 - k_3, G) e^{-i (k_1 - k_2) \cdot \sigma_x\hat{x}/2 - i (k_3 - k_4) \cdot \sigma_y\hat{y}/2} c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4)$''',
    'def_var': r'''$\langle c_{\alpha,s}^\dagger(k_1) c_{\alpha^{\prime},s^{\prime}}(k_2) \rangle$ : expectation of $c_{\alpha,s}^\dagger(k_1) c_{\alpha^{\prime},s^{\prime}}(k_2)$ over the many-body ground states   
$\hat{H}^{int,HF}$ : Interaction term after Hartree-Fock approximation  ''',
    'answer':r'''$\hat{H}^{int,HF,V_{pp}} = \frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_2 - k_1 + k_4 - k_3, G) e^{-i (k_1 - k_2) \cdot \sigma_x\hat{x}/2 - i (k_3 - k_4) \cdot \sigma_y\hat{y}/2} \times \Big[\langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) + \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) - \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle - \langle c_{p_x,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_x,s}(k_2) - \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_x,s}(k_2) \rangle c_{p_x,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) + \langle c_{p_x,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_x,s}(k_2) \rangle \Big]$''',
'source': {'1106.6060.tex':[(221,267)]}

}


In [63]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to perform a Hartree-Fock approximation to expand the interaction term, the fourth term in $\hat{H}^{int}$.  
You should use Wick's theorem to expand the four-fermion term in the fourth term in $\hat{H}^{int}$ into quadratic terms. You should strictly follow the EXAMPLE below to expand using Wick's theorem, select the correct EXAMPLE by noticing the order of four term product with and without ${}^\dagger$, and be extremely cautious about the order of the index and sign before each term.  
You should only preserve the normal terms. Here, the normal terms mean the product of a creation operator and an annihilation operator.  
You should recall that the fourth term is $\frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_2 - k_1 + k_4 - k_3, G) e^{-i (k_1 - k_2) \cdot \sigma_x\hat{x}/2 - i (k_3 - k_4) \cdot \sigma_y\hat{y}/2} c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}

In [64]:
kwargs_list.append(kwargs)

## Extract quadratic term [first term]

In [65]:
kwargs = {
    'task':"Extract quadratic term",
    'HF_symbol': r'the first term in $\hat{H}^{int,HF}$',
    'bilinear_op': r'$c^\dagger c$',
    'HF_2_symbol': r'the first term in $\hat{H}^{int,HF,2}$',
    'expression_HF': r'''the first term in $\hat{H}^{int,HF}$ is $\frac{U_d}{N}\sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \big[ \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4)+ \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) \rangle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2)- \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \rangle \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) \rangle- \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\downarrow}(k_4) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\uparrow}(k_2)- \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\uparrow}(k_2) \rangle c_{d,\uparrow}^\dagger(k_1) c_{d,\downarrow}(k_4)+ \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\downarrow}(k_4) \rangle \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\uparrow}(k_2) \rangle \big]$''',
    'def_var': r'''$\hat{H}^{int,HF,2}$ : the quadratic terms in $\hat{H}^{int,HF}$''',
    'answer':r'''$$\hat{H}^{int,HF,U_d,2}= \frac{U_d}{N}\sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \big[ \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4)
+ \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) \rangle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2)
- \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\downarrow}(k_4) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\uparrow}(k_2)
- \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\uparrow}(k_2) \rangle c_{d,\uparrow}^\dagger(k_1) c_{d,\downarrow}(k_4) \big]$$''',
'source': {'1106.6060.tex':[(221,267)]}

}


In [66]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to extract the quadratic terms in the the first term in $\hat{H}^{int,HF}$.  
The quadratic terms mean terms that are proportional to $c^\dagger c$, which excludes terms that are solely expectations or products of expectations.  
You should only preserve the quadratic terms in the first term in $\hat{H}^{int,HF}$, denoted as the first term in $\hat{H}^{int,HF,2}$.  
You should recall that the first term in $\hat{H}^{int,HF}$ is $\frac{U_d}{N}\sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \big[ \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4)+ \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) \rangle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2)- \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \rangle \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) \rangle- \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\downarrow}(k_4) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d

In [67]:
kwargs_list.append(kwargs)

## Extract quadratic term [second term]

In [68]:
kwargs = {
    'task':"Extract quadratic term",
    'HF_symbol': r'the second term in $\hat{H}^{int,HF}$',
    'bilinear_op': r'$c^\dagger c$',
    'HF_2_symbol': r'the second term in $\hat{H}^{int,HF,2}$',
    'expression_HF': r'''the second term in $\hat{H}^{int,HF}$ is $\frac{U_p}{2N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big\{ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4)+ \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \rangle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2)- \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \rangle- \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\downarrow}(k_4) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\uparrow}(k_2)- \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\downarrow}(k_4)+ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\downarrow}(k_4) \rangle \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\uparrow}(k_2) \rangle + \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) \rangle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4)+ \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4) \rangle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2)- \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) \rangle \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4) \rangle- \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\downarrow}(k_4) \rangle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\uparrow}(k_2)- \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\uparrow}(k_2) \rangle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\downarrow}(k_4)+ \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\downarrow}(k_4) \rangle \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\uparrow}(k_2) \rangle \Big\}$''',
    'def_var': r'''$\hat{H}^{int,HF,2}$ : the quadratic terms in $\hat{H}^{int,HF}$''',
    'answer':r'''$$\hat{H}^{int,HF,U_p,2} = \frac{U_p}{2N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big\{ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4)
+ \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \rangle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2)
- \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\downarrow}(k_4) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\uparrow}(k_2)
- \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\downarrow}(k_4)
+ \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) \rangle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4)
+ \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4) \rangle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2)
- \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\downarrow}(k_4) \rangle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\uparrow}(k_2)
- \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\uparrow}(k_2) \rangle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\downarrow}(k_4) \Big\}.$$''',
'source': {'1106.6060.tex':[(221,267)]}

}


In [69]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to extract the quadratic terms in the the second term in $\hat{H}^{int,HF}$.  
The quadratic terms mean terms that are proportional to $c^\dagger c$, which excludes terms that are solely expectations or products of expectations.  
You should only preserve the quadratic terms in the second term in $\hat{H}^{int,HF}$, denoted as the second term in $\hat{H}^{int,HF,2}$.  
You should recall that the second term in $\hat{H}^{int,HF}$ is $\frac{U_p}{2N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big\{ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4)+ \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \rangle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2)- \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \rangle- \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\downarrow}(k_4) \rangle

In [70]:
kwargs_list.append(kwargs)

## Extract quadratic term [third term]

In [71]:
kwargs = {
    'task':"Extract quadratic term",
    'HF_symbol': r'the third term in $\hat{H}^{int,HF}$',
    'bilinear_op': r'$c^\dagger c$',
    'HF_2_symbol': r'the third term in $\hat{H}^{int,HF,2}$',
    'expression_HF': r'''the third term in $\hat{H}^{int,HF}$ is $2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[\langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right) + \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)- \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \rangle \cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)- \langle c_{d,s}^\dagger(k_1) c_{p_x,s^{\prime}}(k_4) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)- \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle c_{d,s}^\dagger(k_1) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)+ \langle c_{d,s}^\dagger(k_1) c_{p_x,s^{\prime}}(k_4) \rangle \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle \cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)+ \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)+ \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)- \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle \cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)- \langle c_{d,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)- \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle c_{d,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)+ \langle c_{d,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle \cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)\Big]$''',
    'def_var': r'''$\hat{H}^{int,HF,2}$ : the quadratic terms in $\hat{H}^{int,HF}$''',
    'answer':r'''$\hat{H}^{int,HF,V_{pd},2} = 2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[\langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right) + \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)- \langle c_{d,s}^\dagger(k_1) c_{p_x,s^{\prime}}(k_4) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)- \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle c_{d,s}^\dagger(k_1) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)+ \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)+ \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)- \langle c_{d,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)- \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle c_{d,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)\Big]$''',
'source': {'1106.6060.tex':[(221,267)]}

}


In [72]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to extract the quadratic terms in the the third term in $\hat{H}^{int,HF}$.  
The quadratic terms mean terms that are proportional to $c^\dagger c$, which excludes terms that are solely expectations or products of expectations.  
You should only preserve the quadratic terms in the third term in $\hat{H}^{int,HF}$, denoted as the third term in $\hat{H}^{int,HF,2}$.  
You should recall that the third term in $\hat{H}^{int,HF}$ is $2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[\langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right) + \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)- \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \rangle \cos\left

In [73]:
kwargs_list.append(kwargs)

## Extract quadratic term [fourth term]

In [74]:
kwargs = {
    'task':"Extract quadratic term",
    'HF_symbol': r'the fourth term in $\hat{H}^{int,HF}$',
    'bilinear_op': r'$c^\dagger c$',
    'HF_2_symbol': r'the fourth term in $\hat{H}^{int,HF,2}$',
    'expression_HF': r'''the fourth term in $\hat{H}^{int,HF}$ is $\frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_2 - k_1 + k_4 - k_3, G) e^{-i (k_1 - k_2) \cdot \sigma_x\hat{x}/2 - i (k_3 - k_4) \cdot \sigma_y\hat{y}/2} \times \Big[\langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) + \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) - \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle - \langle c_{p_x,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_x,s}(k_2) - \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_x,s}(k_2) \rangle c_{p_x,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) + \langle c_{p_x,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_x,s}(k_2) \rangle \Big]$''',
    'def_var': r'''$\hat{H}^{int,HF,2}$ : the quadratic terms in $\hat{H}^{int,HF}$''',
    'answer':r'''$\hat{H}^{int,HF,V_{pp},2} = \frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_2 - k_1 + k_4 - k_3, G) e^{-i (k_1 - k_2) \cdot \sigma_x\hat{x}/2 - i (k_3 - k_4) \cdot \sigma_y\hat{y}/2} \times \Big[ \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) + \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) - \langle c_{p_x,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_x,s}(k_2) - \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_x,s}(k_2) \rangle c_{p_x,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \Big]$''',
'source': {'1106.6060.tex':[(221,267)]}

}


In [75]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to extract the quadratic terms in the the fourth term in $\hat{H}^{int,HF}$.  
The quadratic terms mean terms that are proportional to $c^\dagger c$, which excludes terms that are solely expectations or products of expectations.  
You should only preserve the quadratic terms in the fourth term in $\hat{H}^{int,HF}$, denoted as the fourth term in $\hat{H}^{int,HF,2}$.  
You should recall that the fourth term in $\hat{H}^{int,HF}$ is $\frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_2 - k_1 + k_4 - k_3, G) e^{-i (k_1 - k_2) \cdot \sigma_x\hat{x}/2 - i (k_3 - k_4) \cdot \sigma_y\hat{y}/2} \times \Big[\langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) + \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) - \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle \langle c_{p_y,s^{\prime}}^\dagger(

In [76]:
kwargs_list.append(kwargs)

## Hartree term only [first term]

In [77]:
kwargs = {
    'task':"Hartree term only",
    'HF_2_symbol': r'the first term in $\hat{H}^{int,HF,2}$',
    'expval_Hartree':r'$\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$',
    'Hartree_2_symbol': r'the first term in $\hat{H}^{int,Hartree}$',
    'expression_HF': r'''the first term in $\hat{H}^{int,HF,2}$ is $\frac{U_d}{N}\sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \big[ \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4)+ \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) \rangle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2)- \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\downarrow}(k_4) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\uparrow}(k_2)- \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\uparrow}(k_2) \rangle c_{d,\uparrow}^\dagger(k_1) c_{d,\downarrow}(k_4) \big]$''',
    'answer':r'''$\frac{U_d}{N} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[ \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) + \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) \rangle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \Big]$''',
'source': {'1106.6060.tex':[(177,267)]}

}


In [78]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to keep only the Hartree term in the first term in $\hat{H}^{int,HF,2}$.  
Here, Hartree term only means that only the expected value in the form $\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$ (Note that the two indices are the same) should be the preserved. All other expected value terms should be dropped.
You should recall that the first term in $\hat{H}^{int,HF,2}$ is $\frac{U_d}{N}\sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \big[ \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4)+ \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) \rangle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2)- \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\downarrow}(k_4) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\uparrow}(k_2)- \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\uparrow}(k_2) \rangle c_{d,\uparrow}^\dagger(k_1) c_{d,\downarrow}(k_4) \big]$  
Return the simplified Hami

In [79]:
kwargs_list.append(kwargs)

## Hartree term only [second term]

In [80]:
kwargs = {
    'task':"Hartree term only",
    'HF_2_symbol': r'the second term in $\hat{H}^{int,HF,2}$',
    'expval_Hartree':r'$\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$',
    'Hartree_2_symbol': r'the second term in $\hat{H}^{int,Hartree}$',
    'expression_HF': r'''the second term in $\hat{H}^{int,HF,2}$ is $\frac{U_p}{2N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big\{ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4)+ \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \rangle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2)- \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \rangle- \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\downarrow}(k_4) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\uparrow}(k_2)- \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\downarrow}(k_4)+ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\downarrow}(k_4) \rangle \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\uparrow}(k_2) \rangle + \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) \rangle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4)+ \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4) \rangle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2)- \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) \rangle \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4) \rangle- \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\downarrow}(k_4) \rangle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\uparrow}(k_2)- \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\uparrow}(k_2) \rangle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\downarrow}(k_4)+ \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\downarrow}(k_4) \rangle \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\uparrow}(k_2) \rangle \Big\}.$''',
    'answer':r'''$\frac{U_p}{2N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big\{ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4)
+ \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \rangle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2)
+ \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) \rangle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4)
+ \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4) \rangle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) \Big\}$''',
'source': {'1106.6060.tex':[(177,267)]}

}


In [81]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to keep only the Hartree term in the second term in $\hat{H}^{int,HF,2}$.  
Here, Hartree term only means that only the expected value in the form $\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$ (Note that the two indices are the same) should be the preserved. All other expected value terms should be dropped.
You should recall that the second term in $\hat{H}^{int,HF,2}$ is $\frac{U_p}{2N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big\{ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4)+ \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \rangle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2)- \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \rangle- \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\downarrow}(k_4) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p

In [82]:
kwargs_list.append(kwargs)

## Hartree term only [third term]

In [83]:
kwargs = {
    'task':"Hartree term only",
    'HF_2_symbol': r'the third term in $\hat{H}^{int,HF,2}$',
    'expval_Hartree':r'$\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$',
    'Hartree_2_symbol': r'the third term in $\hat{H}^{int,Hartree}$',
    'expression_HF': r'''the third term in $\hat{H}^{int,HF,2}$ is $2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[\langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right) + \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)- \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \rangle \cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)- \langle c_{d,s}^\dagger(k_1) c_{p_x,s^{\prime}}(k_4) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)- \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle c_{d,s}^\dagger(k_1) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)+ \langle c_{d,s}^\dagger(k_1) c_{p_x,s^{\prime}}(k_4) \rangle \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle \cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)+ \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)+ \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)- \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle \cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)- \langle c_{d,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)- \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle c_{d,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)+ \langle c_{d,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{d,s}(k_2) \rangle \cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)\Big].$''',
    'answer':r'''$\hat{H}^{int,Hartree,V_{pd}} = 2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[
\langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right) 
+ \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)
+ \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)
+ \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right) 
\Big]$''',
'source': {'1106.6060.tex':[(177,267)]}

}


In [84]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to keep only the Hartree term in the third term in $\hat{H}^{int,HF,2}$.  
Here, Hartree term only means that only the expected value in the form $\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$ (Note that the two indices are the same) should be the preserved. All other expected value terms should be dropped.
You should recall that the third term in $\hat{H}^{int,HF,2}$ is $2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[\langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right) + \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)- \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \rangle \cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\rig

In [85]:
kwargs_list.append(kwargs)

## Hartree term only [fourth term]

In [86]:
kwargs = {
    'task':"Hartree term only",
    'HF_2_symbol': r'the fourth term in $\hat{H}^{int,HF,2}$',
    'expval_Hartree':r'$\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$',
    'Hartree_2_symbol': r'the fourth term in $\hat{H}^{int,Hartree}$',
    'expression_HF': r'''the fourth term in $\hat{H}^{int,HF,2}$ is $\frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_2 - k_1 + k_4 - k_3, G) e^{-i (k_1 - k_2) \cdot \sigma_x\hat{x}/2 - i (k_3 - k_4) \cdot \sigma_y\hat{y}/2} \times \Big[\langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) + \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) - \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle - \langle c_{p_x,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_x,s}(k_2) - \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_x,s}(k_2) \rangle c_{p_x,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) + \langle c_{p_x,s}^\dagger(k_1) c_{p_y,s^{\prime}}(k_4) \rangle \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_x,s}(k_2) \rangle \Big]$''',
    'answer':r'''$\hat{H}^{int,Hartree,V_{pp}} = \frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_2 - k_1 + k_4 - k_3, G) e^{-i (k_1 - k_2) \cdot \sigma_x\hat{x}/2 - i (k_3 - k_4) \cdot \sigma_y\hat{y}/2} \times \Big[ \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) + \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \Big].
$''',
'source': {'1106.6060.tex':[(177,267)]}

}


In [87]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to keep only the Hartree term in the fourth term in $\hat{H}^{int,HF,2}$.  
Here, Hartree term only means that only the expected value in the form $\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$ (Note that the two indices are the same) should be the preserved. All other expected value terms should be dropped.
You should recall that the fourth term in $\hat{H}^{int,HF,2}$ is $\frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_2 - k_1 + k_4 - k_3, G) e^{-i (k_1 - k_2) \cdot \sigma_x\hat{x}/2 - i (k_3 - k_4) \cdot \sigma_y\hat{y}/2} \times \Big[\langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) + \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) - \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle 

In [88]:
kwargs_list.append(kwargs)

## Swap the index only [first term]

In [89]:
kwargs = {
    'task':"Swap the index only",
    'HF_2_symbol': r'the first term in $\hat{H}^{int,HF,2}$',
    'expval': r'$\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$',
    'expression_Hartree_1':r'$\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)$',
    'expression_Hartree_2':r'$\langle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)\rangle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2)$',
    'relabel': '$k_1 \leftrightarrow k_3$ and $k_2 \leftrightarrow k_4$',
    'Unsummed_Indices': r'$\uparrow$ and $\downarrow$', 
    'expression_HF_2':r'''the first term in $\hat{H}^{int,HF,2}$ in $\frac{U_d}{N} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[ \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) + \langle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) \rangle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \Big]$''',
    'answer':r'''$\hat{H}^{int,Hartree,U_d} = \frac{U_d}{N} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[ \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) + \langle c_{d,\downarrow}^\dagger(k_1) c_{d,\downarrow}(k_2) \rangle c_{d,\uparrow}^\dagger(k_3) c_{d,\uparrow}(k_4) \Big].$''',
'source': {'1106.6060.tex':[(177,267)]}

}


In [90]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the quadratic term the first term in $\hat{H}^{int,HF,2}$ through relabeling the index.  
The logic is that the expected value ($\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$) in the first Hartree term ($\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)$) has the same form as the quadratic operators in the second Hartree term ($\langle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)\rangle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2)$), and vice versa. The same applies to the Fock term.  
Namely, a replacement of $k_1 \leftrightarrow k_3$ and $k_2 \leftrightarrow k_4$ is applied to ONLY the second Hartree or Fock term. You should not swap any index that is not in the summation, which includes $\uparrow$ and $\downarrow$.  
This means, if you relabel the index by swapping the index in the "expected value" and "quadratic operators" in the second Hartr

In [91]:
kwargs_list.append(kwargs)

## Swap the index only [second term]

In [92]:
kwargs = {
    'task':"Swap the index only",
    'HF_2_symbol': r'the second term in $\hat{H}^{int,HF,2}$',
    'expval': r'$\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$',
    'expression_Hartree_1':r'$\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)$',
    'expression_Hartree_2':r'$\langle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)\rangle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2)$',
    'relabel': '$k_1 \leftrightarrow k_3$ and $k_2 \leftrightarrow k_4$',
    'Unsummed_Indices': r'$\uparrow$ and $\downarrow$', 
    'expression_HF_2':r'''the second term in $\hat{H}^{int,HF,2}$ is $\frac{U_p}{2N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big\{ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4)+ \langle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4) \rangle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2)+ \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) \rangle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4)+ \langle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4) \rangle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) \Big\}.$''',
    'answer':r''' $\hat{H}^{int,Hartree,U_p} = \frac{U_p}{2N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big\{ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4)+ \langle c_{p_x,\downarrow}^\dagger(k_1) c_{p_x,\downarrow}(k_2) \rangle c_{p_x,\uparrow}^\dagger(k_3) c_{p_x,\uparrow}(k_4)+ \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) \rangle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4)+ \langle c_{p_y,\downarrow}^\dagger(k_1) c_{p_y,\downarrow}(k_2) \rangle c_{p_y,\uparrow}^\dagger(k_3) c_{p_y,\uparrow}(k_4) \Big\}$''',
'source': {'1106.6060.tex':[(177,267)]}

}


In [93]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the quadratic term the second term in $\hat{H}^{int,HF,2}$ through relabeling the index.  
The logic is that the expected value ($\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$) in the first Hartree term ($\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)$) has the same form as the quadratic operators in the second Hartree term ($\langle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)\rangle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2)$), and vice versa. The same applies to the Fock term.  
Namely, a replacement of $k_1 \leftrightarrow k_3$ and $k_2 \leftrightarrow k_4$ is applied to ONLY the second Hartree or Fock term. You should not swap any index that is not in the summation, which includes $\uparrow$ and $\downarrow$.  
This means, if you relabel the index by swapping the index in the "expected value" and "quadratic operators" in the second Hart

In [94]:
kwargs_list.append(kwargs)

## Swap the index only [third term]

In [95]:
kwargs = {
    'task':"Swap the index only",
    'HF_2_symbol': r'the third term in $\hat{H}^{int,HF,2}$',
    'expval': r'$\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$',
    'expression_Hartree_1':r'$\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)$',
    'expression_Hartree_2':r'$\langle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)\rangle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2)$',
    'relabel': '$k_1 \leftrightarrow k_3$ and $k_2 \leftrightarrow k_4$',
    'Unsummed_Indices': r'$\uparrow$ and $\downarrow$', 
    'expression_HF_2':r'''the third term in $\hat{H}^{int,HF,2}$ is $2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[\langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right) + \langle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right)+ \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)+ \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{d,s}^\dagger(k_1) c_{d,s}(k_2)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right) \Big]$''',
    'answer':r'''$\hat{H}^{int,Hartree,V_{pd}} = 2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[\langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right) + \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle c_{d,s^{\prime}}^\dagger(k_3) c_{d,s^{\prime}}(k_4)\cos\left(\frac{k_1-k_2}{2}\cdot \hat{x}\right)+ \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)+ \langle c_{p_y,s}^\dagger(k_1) c_{p_y,s}(k_2) \rangle c_{d,s^{\prime}}^\dagger(k_3) c_{d,s^{\prime}}(k_4)\cos\left(\frac{k_1-k_2}{2}\cdot \hat{y}\right) \Big].$''',
'source': {'1106.6060.tex':[(177,267)]}

}


In [96]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the quadratic term the third term in $\hat{H}^{int,HF,2}$ through relabeling the index.  
The logic is that the expected value ($\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$) in the first Hartree term ($\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)$) has the same form as the quadratic operators in the second Hartree term ($\langle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)\rangle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2)$), and vice versa. The same applies to the Fock term.  
Namely, a replacement of $k_1 \leftrightarrow k_3$ and $k_2 \leftrightarrow k_4$ is applied to ONLY the second Hartree or Fock term. You should not swap any index that is not in the summation, which includes $\uparrow$ and $\downarrow$.  
This means, if you relabel the index by swapping the index in the "expected value" and "quadratic operators" in the second Hartr

In [97]:
kwargs_list.append(kwargs)

## Swap the index only [fourth term]

In [98]:
kwargs = {
    'task':"Swap the index only",
    'HF_2_symbol': r'the fourth term in $\hat{H}^{int,HF,2}$',
    'expval': r'$\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$',
    'expression_Hartree_1':r'$\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)$',
    'expression_Hartree_2':r'$\langle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)\rangle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2)$',
    'relabel': '$k_1 \leftrightarrow k_3$ and $k_2 \leftrightarrow k_4$',
    'Unsummed_Indices': r'$\uparrow$ and $\downarrow$', 
    'expression_HF_2':r'''the fourth term in $\hat{H}^{int,HF,2}$ is $\frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_2 - k_1 + k_4 - k_3, G) e^{-i (k_1 - k_2) \cdot \sigma_x\hat{x}/2 - i (k_3 - k_4) \cdot \sigma_y\hat{y}/2} \times \Big[ \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) + \langle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) \rangle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \Big]$''',
    'answer':r'''$\frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_2 - k_1 + k_4 - k_3, G) e^{-i (k_1 - k_2) \cdot \sigma_x\hat{x}/2 - i (k_3 - k_4) \cdot \sigma_y\hat{y}/2} \times \Big[ \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) + \langle c_{p_y,s}^\dagger(k_1) c_{p_y,s}(k_2) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \Big].$''',
'source': {'1106.6060.tex':[(177,267)]}

}


In [99]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the quadratic term the fourth term in $\hat{H}^{int,HF,2}$ through relabeling the index.  
The logic is that the expected value ($\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle$) in the first Hartree term ($\langle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2) \rangle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)$) has the same form as the quadratic operators in the second Hartree term ($\langle c_{\alpha_2,s_2}^\dagger(k_3) c_{\alpha_2,s_2}(k_4)\rangle c_{\alpha_1,s_1}^\dagger(k_1) c_{\alpha_1,s_1}(k_2)$), and vice versa. The same applies to the Fock term.  
Namely, a replacement of $k_1 \leftrightarrow k_3$ and $k_2 \leftrightarrow k_4$ is applied to ONLY the second Hartree or Fock term. You should not swap any index that is not in the summation, which includes $\uparrow$ and $\downarrow$.  
This means, if you relabel the index by swapping the index in the "expected value" and "quadratic operators" in the second Hart

In [100]:
kwargs_list.append(kwargs)

## Reduce momentum in Hartree term (momentum in BZ) [first term] 

In [101]:
kwargs = {
    'task':"Reduce momentum in Hartree term (momentum in BZ)",
    'Hartree_2_symbol': r'the first term in $\hat{H}^{int,Hartree,2}$',
    'expval': r'$\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle$',
    'expval_id':r'$\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle=\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle \delta_{k_i,k_j}$',
    'expression_Hartree':r'the first term in $\hat{H}^{int,Hartree,2} = \frac{U_d}{N} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[ \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_2) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_4) + \langle c_{d,\downarrow}^\dagger(k_1) c_{d,\downarrow}(k_2) \rangle c_{d,\uparrow}^\dagger(k_3) c_{d,\uparrow}(k_4) \Big]$',
    'answer':r'''$$\hat{H}^{int,Hartree,U_d} = \frac{U_d}{N} \sum_{k_1, k_3} \Big[ \langle c_{d,\uparrow}^\dagger(k_1) c_{d,\uparrow}(k_1) \rangle c_{d,\downarrow}^\dagger(k_3) c_{d,\downarrow}(k_3) + \langle c_{d,\downarrow}^\dagger(k_1) c_{d,\downarrow}(k_1) \rangle c_{d,\uparrow}^\dagger(k_3) c_{d,\uparrow}(k_3) \Big].$$''',
'source': {'1106.6060.tex':[(177,267)]}

}


In [102]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the Hartree term, the first term in $\hat{H}^{int,Hartree,2}$, by reducing the momentum inside the expected value $\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle$.  
The expected value $\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle$ is only nonzero when the two momenta $k_i,k_j$ are the same, namely, $\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle=\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle \delta_{k_i,k_j}$.  
You should use the property of Kronecker delta function $\delta_{k_i,k_j}$ to reduce one momentum $k_i$.
Once you reduce one momentum inside the expected value $\langle\dots\rangle$. You will also notice the total momentum conservation will reduce another momentum in the quadratic term. Therefore, you should end up with only two momenta left in the summation.  
You should follow the EXAMPLE below to reduce one momentum in the Hartree term, and another moment

In [103]:
kwargs_list.append(kwargs)

## Reduce momentum in Hartree term (momentum in BZ) [second term] 

In [104]:
kwargs = {
    'task':"Reduce momentum in Hartree term (momentum in BZ)",
    'Hartree_2_symbol': r'the second term in $\hat{H}^{int,Hartree,2}$',
    'expval': r'$\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle$',
    'expval_id':r'$\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle=\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle \delta_{k_i,k_j}$',
    'expression_Hartree':r'the second term in $\hat{H}^{int,Hartree,2} = \frac{U_p}{2N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big\{ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_2) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_4)+ \langle c_{p_x,\downarrow}^\dagger(k_1) c_{p_x,\downarrow}(k_2) \rangle c_{p_x,\uparrow}^\dagger(k_3) c_{p_x,\uparrow}(k_4)+ \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_2) \rangle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_4)+ \langle c_{p_y,\downarrow}^\dagger(k_1) c_{p_y,\downarrow}(k_2) \rangle c_{p_y,\uparrow}^\dagger(k_3) c_{p_y,\uparrow}(k_4) \Big\}$',
    'answer':r'''$$\hat{H}^{int,Hartree,U_p} = \frac{U_p}{2N} \sum_{k_1, k_3} \Big\{ \langle c_{p_x,\uparrow}^\dagger(k_1) c_{p_x,\uparrow}(k_1) \rangle c_{p_x,\downarrow}^\dagger(k_3) c_{p_x,\downarrow}(k_3)+ \langle c_{p_x,\downarrow}^\dagger(k_1) c_{p_x,\downarrow}(k_1) \rangle c_{p_x,\uparrow}^\dagger(k_3) c_{p_x,\uparrow}(k_3)+ \langle c_{p_y,\uparrow}^\dagger(k_1) c_{p_y,\uparrow}(k_1) \rangle c_{p_y,\downarrow}^\dagger(k_3) c_{p_y,\downarrow}(k_3)+ \langle c_{p_y,\downarrow}^\dagger(k_1) c_{p_y,\downarrow}(k_1) \rangle c_{p_y,\uparrow}^\dagger(k_3) c_{p_y,\uparrow}(k_3) \Big\}.$$''',
'source': {'1106.6060.tex':[(177,267)]}

}


In [105]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the Hartree term, the second term in $\hat{H}^{int,Hartree,2}$, by reducing the momentum inside the expected value $\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle$.  
The expected value $\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle$ is only nonzero when the two momenta $k_i,k_j$ are the same, namely, $\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle=\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle \delta_{k_i,k_j}$.  
You should use the property of Kronecker delta function $\delta_{k_i,k_j}$ to reduce one momentum $k_i$.
Once you reduce one momentum inside the expected value $\langle\dots\rangle$. You will also notice the total momentum conservation will reduce another momentum in the quadratic term. Therefore, you should end up with only two momenta left in the summation.  
You should follow the EXAMPLE below to reduce one momentum in the Hartree term, and another momen

In [106]:
kwargs_list.append(kwargs)

## Reduce momentum in Hartree term (momentum in BZ) [third term] 

In [107]:
kwargs = {
    'task':"Reduce momentum in Hartree term (momentum in BZ)",
    'Hartree_2_symbol': r'the third term in $\hat{H}^{int,Hartree,2}$',
    'expval': r'$\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle$',
    'expval_id':r'$\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle=\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle \delta_{k_i,k_j}$',
    'expression_Hartree':r'the third term in $\hat{H}^{int,Hartree,2} = 2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_2,k_3,k_4} \sum_{G} \delta(k_1-k_2+k_3-k_4,G) \Big[\langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{x}\right) + \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle c_{d,s^{\prime}}^\dagger(k_3) c_{d,s^{\prime}}(k_4)\cos\left(\frac{k_1-k_2}{2}\cdot \hat{x}\right)+ \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4)\cos\left(\frac{k_3-k_4}{2}\cdot \hat{y}\right)+ \langle c_{p_y,s}^\dagger(k_1) c_{p_y,s}(k_2) \rangle c_{d,s^{\prime}}^\dagger(k_3) c_{d,s^{\prime}}(k_4)\cos\left(\frac{k_1-k_2}{2}\cdot \hat{y}\right) \Big]',
    'answer':r'''$\hat{H}^{int,Hartree,V_{pd}} = 2V_{pd} \sum_{s, s^{\prime}} \frac{1}{N} \sum_{k_1,k_3} \Big[
\langle c_{d,s}^\dagger(k_1) c_{d,s}(k_1) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_3) 
+ \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_1) \rangle c_{d,s^{\prime}}^\dagger(k_3) c_{d,s^{\prime}}(k_3)
+ \langle c_{d,s}^\dagger(k_1) c_{d,s}(k_1) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_3)
+ \langle c_{p_y,s}^\dagger(k_1) c_{p_y,s}(k_1) \rangle c_{d,s^{\prime}}^\dagger(k_3) c_{d,s^{\prime}}(k_3) 
\Big].$''',
'source': {'1106.6060.tex':[(177,267)]}

}


In [108]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the Hartree term, the third term in $\hat{H}^{int,Hartree,2}$, by reducing the momentum inside the expected value $\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle$.  
The expected value $\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle$ is only nonzero when the two momenta $k_i,k_j$ are the same, namely, $\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle=\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle \delta_{k_i,k_j}$.  
You should use the property of Kronecker delta function $\delta_{k_i,k_j}$ to reduce one momentum $k_i$.
Once you reduce one momentum inside the expected value $\langle\dots\rangle$. You will also notice the total momentum conservation will reduce another momentum in the quadratic term. Therefore, you should end up with only two momenta left in the summation.  
You should follow the EXAMPLE below to reduce one momentum in the Hartree term, and another moment

In [109]:
kwargs_list.append(kwargs)

## Reduce momentum in Hartree term (momentum in BZ) [fourth term] 

In [110]:
kwargs = {
    'task':"Reduce momentum in Hartree term (momentum in BZ)",
    'Hartree_2_symbol': r'the fourth term in $\hat{H}^{int,Hartree,2}$',
    'expval': r'$\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle$',
    'expval_id':r'$\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle=\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle \delta_{k_i,k_j}$',
    'expression_Hartree':r'the fourth term in $\hat{H}^{int,Hartree,2} = \frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_2, k_3, k_4} \sum_{G} \delta(k_2 - k_1 + k_4 - k_3, G) e^{-i (k_1 - k_2) \cdot \sigma_x\hat{x}/2 - i (k_3 - k_4) \cdot \sigma_y\hat{y}/2} \times \Big[ \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_2) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_4) + \langle c_{p_y,s}^\dagger(k_1) c_{p_y,s}(k_2) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_4) \Big]',
    'answer':r'''$\hat{H}^{int,Hartree, V_{pp}} = \frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_3} \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_1) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_3) + \langle c_{p_y,s}^\dagger(k_1) c_{p_y,s}(k_1) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_3).$''',
'source': {'1106.6060.tex':[(177,267)]}

}


In [111]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the Hartree term, the fourth term in $\hat{H}^{int,Hartree,2}$, by reducing the momentum inside the expected value $\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle$.  
The expected value $\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle$ is only nonzero when the two momenta $k_i,k_j$ are the same, namely, $\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle=\langle c_{\alpha_i,s_i}^\dagger(k_i) c_{\alpha_j,s_j}(k_j) \rangle \delta_{k_i,k_j}$.  
You should use the property of Kronecker delta function $\delta_{k_i,k_j}$ to reduce one momentum $k_i$.
Once you reduce one momentum inside the expected value $\langle\dots\rangle$. You will also notice the total momentum conservation will reduce another momentum in the quadratic term. Therefore, you should end up with only two momenta left in the summation.  
You should follow the EXAMPLE below to reduce one momentum in the Hartree term, and another momen

In [112]:
kwargs_list.append(kwargs)

## Mathematical simplify: reduce index

In [113]:
kwargs = {
    'task':"Mathematical simplify: reduce index",
    'symbol': r'$\hat{H}^{int,Hartree, V_{pp}}$',
    'index': r'$\sigma_x$ and $\sigma_y$, where $\sigma_x=\pm 1$, and $\sigma_y=\pm 1$',
    'expressoin_symbol':r'''$\hat{H}^{int,Hartree, V_{pp}} = \frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_3} \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_1) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_3) + \langle c_{p_y,s}^\dagger(k_1) c_{p_y,s}(k_1) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_3).$''',

    'answer':r'''$$\hat{H}^{int,Hartree, V_{pp}} = \frac{4V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{k_1, k_3} \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_1) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_3) + \langle c_{p_y,s}^\dagger(k_1) c_{p_y,s}(k_1) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_3).$$''',
'source': {}
}


In [114]:
print_save(generate_prompt(kwargs,prompt_dict=prompt_dict())['content'])

You will be instructed to simplify the $\hat{H}^{int,Hartree, V_{pp}}$ by performing constant term summation by reducing the unnecessary indices $\sigma_x$ and $\sigma_y$, where $\sigma_x=\pm 1$, and $\sigma_y=\pm 1$.
You should recall that $\hat{H}^{int,Hartree, V_{pp}} = \frac{V_{pp}}{N} \sum_{s, s^{\prime}} \sum_{\sigma_x, \sigma_y} \sum_{k_1, k_3} \langle c_{p_x,s}^\dagger(k_1) c_{p_x,s}(k_1) \rangle c_{p_y,s^{\prime}}^\dagger(k_3) c_{p_y,s^{\prime}}(k_3) + \langle c_{p_y,s}^\dagger(k_1) c_{p_y,s}(k_1) \rangle c_{p_x,s^{\prime}}^\dagger(k_3) c_{p_x,s^{\prime}}(k_3).$.  
Return the simplified Kinetic Hamiltonian $\hat{H}^{int,Hartree, V_{pp}}$.


In [115]:
kwargs_list.append(kwargs)

In [116]:
len(kwargs_list)

33

In [117]:
--

SyntaxError: invalid syntax (3659366440.py, line 1)

## Save jsonl

In [118]:
with open('1106.6060.jsonl', 'w') as f:
    for dict_obj in kwargs_list:
        f.write(json.dumps(dict_obj) + '\n')

# Draft